In [ ]:
import sys, h5py, os
#sys.path.append('/Users/crgalley/Physics/Research/Projects/ReducedBasis/Surrogates/gwsurrogate/')
sys.path.append('/home/scott/Repos/GitRepos/Codes/gwsurrogate/')
import gwsurrogate as gws
from gwsurrogate import surrogateIO

import gwsurrogate.gwtools as gwtools


import lal as lal
import lalsimulation as lalsim

import matplotlib.pyplot as plt
import numpy as np

### convert between text and hdf5 surrogates

### Text 2 text (IO test)

Load a text-based surrogate and save it as another text-based surrogate. This process should be 1-to-1 and onto!

In [ ]:
### Load an existing surrogate
path_to_surrogate = '../tutorial/TutorialSurrogate/EOB_q1_2_NoSpin_Mode22/l2_m2_len12239M_SurID19poly/' # assumes notebook launched from "notebooks" directory
EOBNRv2_sur = gws.EvaluateSingleModeSurrogate(path_to_surrogate)

t,hp,hc = EOBNRv2_sur(1.2)
print EOBNRv2_sur.time().shape
print hp.shape

In [ ]:
### create a bare surrogate directory 
new_surrogate = surrogateIO.TextSurrogateWrite('new_surrogate5/')
print("surrogate ID is the directory %s"%new_surrogate.SurrogateID)

In [ ]:
new_surrogate.write_text?

In [ ]:
### Use surrogate data from EOBNRv2_sur to create a new surrogate
new_surrogate.write_text(EOBNRv2_sur.time_info, EOBNRv2_sur.B,EOBNRv2_sur.eim_indices,\
              EOBNRv2_sur.greedy_points,EOBNRv2_sur.fit_interval,EOBNRv2_sur.affine_map,\
              EOBNRv2_sur.fitparams_amp,EOBNRv2_sur.fitparams_phase,EOBNRv2_sur.fitparams_norm,\
              EOBNRv2_sur.V,EOBNRv2_sur.R,EOBNRv2_sur.fit_type_phase,EOBNRv2_sur.fit_type_amp,\
              EOBNRv2_sur.fit_type_norm,EOBNRv2_sur.parameterization,EOBNRv2_sur.surrogate_mode_type)

In [ ]:
written_surr = gws.EvaluateSingleModeSurrogate('new_surrogate5/')

In [ ]:
gws.CompareSingleModeSurrogate(written_surr,EOBNRv2_sur)

### Text 2 hdf5 to text

Save a surrogate as an hdf5 file, then back to text

In [ ]:
x = surrogateIO.TextSurrogateRead(path_to_surrogate)
x.__dict__.keys()

In [ ]:
writeh5 = surrogateIO.H5Surrogate('tmp1.h5','w')
#isinstance(written_surr,surrogateIO.TextSurrogateRead)
writeh5.required

In [ ]:
#tmpdict = writeh5.prepare_data(written_surr.__dict__)
#writeh5.print_required()
#writeh5.check_required(x.__dict__)
writeh5.write_h5(x.__dict__,subdir='l2_m2')

In [ ]:
surrogateIO.H5Surrogate._surrogate_ID_grp

In [ ]:
surrogateIO.H5Surrogate('tmp1.h5',mode='r', subdir='', closeQ=True)

In [ ]:
fp = h5py.File('tmp1.h5','r')
#os.listdir('.')
#fp.close()
fp['l2_m2']['t_units']

In [ ]:
EOBNRv2_sur_h5 = gws.EvaluateSingleModeSurrogate('tmp1.h5',subdir='l2_m2')

In [ ]:
t,hp,hc = EOBNRv2_sur(1.2)
t_h5,hp_h5,hc_h5 = EOBNRv2_sur_h5(1.2)

print EOBNRv2_sur.time().shape
print hp.shape
print EOBNRv2_sur_h5.time().shape
print hp_h5.shape

# compare h5 and text surrogates. should be equivalent.
print np.max(np.abs(hp - hp_h5))
print np.max(np.abs(hc - hc_h5))
print np.max(np.abs(t_h5 - t))

In [ ]:
# and a more complete comparison of the data fields and values
gws.CompareSingleModeSurrogate(EOBNRv2_sur_h5,EOBNRv2_sur)

In [ ]:
amp, phase = EOBNRv2_sur.amp_phase(hp + 1j*hc)
amp_h5, phase_h5 = EOBNRv2_sur.amp_phase(hp_h5 + 1j*hc_h5)
gwtools.plot_pretty(t, [hp_h5, hc_h5])
plt.plot(t,amp_h5,'r')
plt.xlabel('t (seconds)')
plt.show()

# Work to convert surrogateimr (matlab) regression data from text to hdf5 format

In [ ]:
### SpEC_q1_9_NoSpin_SingleModes_REF
x='/home/balzani57/Desktop/regression_data/SpEC_q1_9_NoSpin_SingleModes_REF/l2_m2_len2800M_SurID22poly/'
#x='/home/balzani57/Desktop/regression_data/SpEC_q0_0_NoSpin_SingleModes_REF/'
Spec_q = gws.EvaluateSingleModeSurrogate(x)

In [ ]:
### Evaluate text-based surrogate ###
t,hp,hc = Spec_q(3.14)
amp, phase = Spec_q.amp_phase(hp + 1j*hc)
amp, phase = Spec_q.amp_phase(hp + 1j*hc)
gwtools.plot_pretty(t, [hp, hc])
plt.plot(t,amp,'r')
plt.xlabel('t (seconds)')
plt.show()

In [ ]:
### do the conversion to hdf5
y = surrogateIO.TextSurrogateRead(x)

In [ ]:
writeh5 = surrogateIO.H5Surrogate('spec_tmp.h5','w')
writeh5.write_h5(y.__dict__,subdir='l2_m2')

In [ ]:
spec_sur_h5 = gws.EvaluateSingleModeSurrogate('spec_tmp.h5',subdir='l2_m2')

In [ ]:
t,hp,hc = Spec_q(3.14)
t_h5,hp_h5,hc_h5 = spec_sur_h5(3.14)

print Spec_q.time().shape
print hp.shape
print spec_sur_h5.time().shape
print hp_h5.shape

# compare h5 and text surrogates. should be equivalent.
print np.max(np.abs(hp - hp_h5))
print np.max(np.abs(hc - hc_h5))
print np.max(np.abs(t_h5 - t))

In [ ]:
gws.CompareSingleModeSurrogate(spec_sur_h5,Spec_q)

In [ ]:
x='/home/balzani57/Desktop/regression_data/SpEC_q0_0_NoSpin_SingleModes_REF/'
y = gws.CreateManyEvaluateSingleModeSurrogates(x, deg=3, ell_m=None, excluded=None)

In [ ]:
## could loop over modes here, change file mode to append (r+)
writeh5 = surrogateIO.H5Surrogate('spec_tmp3.h5','w')
writeh5.write_h5(y[(2,2)].__dict__,subdir='l2_m2')

### Compare Christians spherical harmonic code to lal  Outcome: all modes up to ell = 5 agree to better than 1e-14 absolute error

In [ ]:
lal.SpinWeightedSphericalHarmonic(theta = 0.0, phi=0.0,s=-2,l=2,m=2)

In [ ]:
l=2
m=1

theta_list = np.linspace(0.0,np.pi,200)
phi_list   = np.linspace(0.0,2.0*np.pi,200)
error_list1 = np.zeros((200,200),dtype=complex)
error_list2 = np.zeros((200,200),dtype=complex)

ii=0
jj=0
for theta in theta_list:
    
    for phi in phi_list:
        lal_spin = lal.SpinWeightedSphericalHarmonic(theta, phi, s=-2, l=l, m=m)
        cr_spin  = gws.sYlm(-2, ll=l, mm=m,theta=theta, phi=phi)
        error_list1[ii,jj] = lal_spin - cr_spin
        error_list2[ii,jj] = (lal_spin - cr_spin)
        jj+=1
        
    ii+=1
    jj=0    
    
    
print np.max(np.abs(error_list1))